In [1]:
from gnatss.ops.posfilter import kalman_filtering
from gnatss.loaders import read_novatel_L1_data_files
from gnatss.loaders import load_gps_solutions

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)


import warnings
warnings.filterwarnings('ignore')

In [2]:
gps_novatel_df = load_gps_solutions(
    files = [
        'tests/data/2022/NCL1/208/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/209/GPS_PPP/GPS_POS_FREED',
        'tests/data/2022/NCL1/210/GPS_PPP/GPS_POS_FREED',
    ],
    is_novatel_l1_data=True,
)
gps_novatel_df

,time,x,y,z,xx,yy,zz
0,712207183.000,-2574893.320,-3681574.160,4512096.640,2.449,2.449,2.449
1,712207184.000,-2574893.280,-3681574.056,4512096.748,2.236,2.236,2.236
2,712207185.000,-2574893.239,-3681573.952,4512096.855,2.000,2.000,2.000
3,712207186.000,-2574893.198,-3681573.848,4512096.963,1.732,1.732,1.732
4,712207187.000,-2574893.158,-3681573.744,4512097.070,1.415,1.414,1.414
...,...,...,...,...,...,...,...
193288,712400473.000,-2575282.934,-3682610.180,4511037.847,0.013,0.014,0.016
193289,712400474.000,-2575283.675,-3682610.145,4511038.077,0.013,0.014,0.016
193290,712400475.000,-2575283.932,-3682610.222,4511037.953,0.013,0.014,0.016
193291,712400476.000,-2575283.703,-3682609.184,4511037.489,0.013,0.014,0.016


In [3]:
inspvaa_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSPVAA.dat'], data_format="INSPVAA")
insstdeva_df = read_novatel_L1_data_files(data_files=['tests/data/2022/NCL1/NCL1_INSSTDEVA.dat'],
                                          data_format="INSSTDEVA")

In [4]:
inspvaa_df

,Week,Seconds,lat,lon,alt,ant_n,ant_e,ant_u,roll,pitch,heading,time
0,2220,400732.750,45.302,-124.966,-28.345,-0.686,0.101,0.241,-3.550,1.676,225.486,712293532.750
1,2220,400732.800,45.302,-124.966,-28.334,-0.638,0.079,0.273,-3.541,1.373,225.573,712293532.800
2,2220,400732.850,45.302,-124.966,-28.323,-0.583,0.053,0.293,-3.183,1.126,225.649,712293532.850
3,2220,400732.900,45.302,-124.966,-28.310,-0.544,0.041,0.295,-2.585,0.941,225.705,712293532.900
4,2220,400732.950,45.302,-124.966,-28.299,-0.512,0.035,0.280,-1.819,0.827,225.733,712293532.950
...,...,...,...,...,...,...,...,...,...,...,...,...
3865082,2220,469048.150,45.302,-124.966,-23.490,-0.973,1.063,-0.116,2.405,-1.053,127.182,712361848.150
3865083,2220,469048.200,45.302,-124.966,-23.496,-0.980,1.089,-0.142,2.849,-0.736,126.521,712361848.200
3865084,2220,469048.250,45.302,-124.966,-23.504,-0.971,1.125,-0.173,3.109,-0.441,125.873,712361848.250
3865085,2220,469048.300,45.302,-124.966,-23.513,-0.954,1.158,-0.209,3.184,-0.162,125.248,712361848.300


In [5]:
insstdeva_df

,Week,Seconds,lat std,lon std,alt std,ant_n std,ant_e std,ant_u std,roll_roll,pitch_pitch,heading_heading,Ext sol stat,Time Since Update,time
0,2220,400732.750,0.500,0.440,0.728,0.018,0.016,0.016,0.030,0.031,0.146,0e0000d3,0,712293532.750
1,2220,400732.800,0.500,0.440,0.728,0.018,0.016,0.016,0.030,0.031,0.146,0e0000d3,0,712293532.800
2,2220,400732.850,0.500,0.440,0.728,0.018,0.016,0.016,0.030,0.031,0.146,0e0000d3,0,712293532.850
3,2220,400732.900,0.500,0.440,0.728,0.018,0.016,0.016,0.030,0.031,0.146,0e0000d3,0,712293532.900
4,2220,400732.950,0.500,0.440,0.728,0.018,0.016,0.016,0.030,0.031,0.146,0e0000d3,0,712293532.950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3865303,2220,469048.150,0.375,0.318,0.741,0.017,0.015,0.021,0.025,0.026,0.126,0e0000d3,0,712361848.150
3865304,2220,469048.200,0.375,0.318,0.741,0.017,0.015,0.021,0.025,0.026,0.126,0e0000d3,0,712361848.200
3865305,2220,469048.250,0.375,0.318,0.741,0.017,0.015,0.021,0.025,0.026,0.126,0e0000d3,0,712361848.250
3865306,2220,469048.300,0.375,0.318,0.741,0.017,0.015,0.021,0.025,0.026,0.126,0e0000d3,0,712361848.300


In [11]:
kalman_filtered_df = kalman_filtering(inspvaa_df, insstdeva_df, gps_novatel_df)[
    ['time', 'x', 'y', 'z', 'xx', 'yy', 'zz']
]


In [12]:
kalman_filtered_df

,time,x,y,z,xx,yy,zz
0,712207205.000,-2574898.373,-3681585.482,4512084.513,-0.287,-0.688,-1.308
1,712207205.050,-2574898.387,-3681585.517,4512084.448,-0.287,-0.688,-1.308
2,712207205.100,-2574898.402,-3681585.551,4512084.382,-0.277,-0.695,-1.297
3,712207205.150,-2574898.415,-3681585.586,4512084.318,-0.267,-0.702,-1.278
4,712207205.200,-2574898.429,-3681585.621,4512084.254,-0.260,-0.706,-1.252
...,...,...,...,...,...,...,...
3865063,712400477.500,-2575284.791,-3682607.964,4511038.023,-1.179,0.632,0.274
3865064,712400477.550,-2575284.850,-3682607.932,4511038.037,-1.184,0.667,0.198
3865065,712400477.600,-2575284.909,-3682607.899,4511038.047,-1.164,0.708,0.095
3865066,712400477.650,-2575284.967,-3682607.864,4511038.052,-1.128,0.740,-0.002


In [13]:
gps_pos_freed_trans = load_gps_solutions(
    files = [
        'tests/data/2022/NCL1/208/posfilter/POS_FREED_TRANS_TWTT',
        'tests/data/2022/NCL1/209/posfilter/POS_FREED_TRANS_TWTT',
        'tests/data/2022/NCL1/210/posfilter/POS_FREED_TRANS_TWTT',
    ],
    # is_novatel_l1_data=True,
)
gps_pos_freed_trans

,time,x,y,z,xx,xy,xz,yx,yy,yz,zx,zy,zz
0,712215465.000,-2575288.225,-3682570.227,4511064.310,0.001,0.000,-0.000,0.000,0.001,-0.000,-0.000,-0.000,0.001
1,712215467.263,-2575286.789,-3682571.528,4511065.018,0.000,0.000,-0.000,0.000,0.001,-0.000,-0.000,-0.000,0.000
2,712215467.383,-2575286.735,-3682571.582,4511065.013,0.000,0.000,-0.000,0.000,0.001,-0.000,-0.000,-0.000,0.001
3,712215467.401,-2575286.727,-3682571.590,4511065.010,0.000,0.000,-0.000,0.000,0.001,-0.000,-0.000,-0.000,0.001
4,712215480.000,-2575279.439,-3682576.054,4511065.580,0.001,0.000,-0.000,0.000,0.001,-0.000,-0.000,-0.000,0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47984,712396373.473,-2575338.610,-3682612.733,4511002.670,0.001,-0.000,-0.000,-0.000,0.001,-0.000,-0.000,-0.000,0.001
47985,712396386.000,-2575336.177,-3682621.936,4510996.736,0.001,-0.000,-0.000,-0.000,0.001,-0.000,-0.000,-0.000,0.001
47986,712396388.251,-2575336.427,-3682622.326,4510995.568,0.000,0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,0.000
47987,712396388.318,-2575336.394,-3682622.347,4510995.578,0.000,0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,0.001


In [ ]:
# Now perform any comparison between the 
# gps_pos_freed_trans and the kalman_filtered_df